In [73]:
polish_airports = [
    "WAW",  # Warsaw Chopin
    "WMI",  # Warsaw Modlin
    "KRK",  # Krakow
    "GDN",  # Gdansk
    "WRO",  # Wroclaw
    "KTW",  # Katowice
    "POZ",  # Poznan
]

In [74]:
summer_destinations_from_poland = [
    # Greece
    "HER",  # Heraklion (Crete)
    "RHO",  # Rhodes
    "SKG",  # Thessaloniki
    "CFU",  # Corfu
    "ZTH",  # Zakynthos
    "KGS",  # Kos
    "CHQ",  # Chania

    # Spain
    "MAD",  # Madrid Barajas
    "BCN",  # Barcelona El Prat
    "AGP",  # Málaga
    "PMI",  # Palma de Mallorca
    "ALC",  # Alicante
    "TFS",  # Tenerife South
    "SVQ",  # Seville
    "IBZ",  # Ibiza
    "VLC",  # Valencia
    "BIO",  # Bilbao
    "TFN",  # Tenerife North
    "SPC",  # La Palma
    "GRX",  # Granada

    # Italy
    "FCO",  # Rome Fiumicino
    "CIA",  # Rome Ciampino
    "MXP",  # Milan Malpensa
    "BGY",  # Bergamo (Orio al Serio)
    "VCE",  # Venice Marco Polo
    "NAP",  # Naples
    "BLQ",  # Bologna
    "PSA",  # Pisa
    "FLR",  # Florence
    "TRN",  # Turin
    "BRI",  # Bari
    "PMO",  # Palermo
    "CTA",  # Catania
    "OLB",  # Olbia
    "AHO",  # Alghero
    "VRN",  # Verona
    "GOA",  # Genoa

    # Croatia
    "SPU",  # Split
    "DBV",  # Dubrovnik
    "ZAD",  # Zadar

    # Cyprus
    "LCA",  # Larnaca
    "PFO",  # Paphos

    # Bulgaria
    "VAR",  # Varna
    "BOJ",  # Burgas
    "SOF",  #Sofia

    # Montenegro
    "TGD",  # Podgorica

    # Albania
    "TIA",  # Tirana

    # Portugal
    "FAO",  # Faro (Algarve)
    "LIS",  # Lisbon
    "OPO",  # Porto
    "FNC",  # Madeira

    # France
    "CDG",  # Paris Charles de Gaulle
    "ORY",  # Paris Orly
    "BVA",  # Paris Beauvais
    "LYS",  # Lyon–Saint-Exupéry
    "NCE",  # Nice Côte d'Azur
    "MRS",  # Marseille Provence
    "BIQ",  # Biarritz Pays Basque

    "KEF",  # Reyjkjavik

    "RMO",  #Moldova
    "BBU",  #Bucharest
    "OTP",  #Bucharest
    "CPH",  #Copenhagen
    "ARN",  #Stocholm
    "GOT",  #Gothenburg
    "ATH",  #Athens
    "BUD",  #Budapest
    "AUH",  #AbuDhabi
    "IST",  #Istanbul
    "SKP",  #Skopje
    "KUT",  #Kutaisi
    "MLA", #Malta
    "AMM", #Amman
    "RAK", #Marakech
    "RBA", #Rabat
    "AGA", #Agadir
    "EIN", #Eindhoven
    "OSL", #Oslo
    "TRF", #Oslo
    "AMS", #Amsterdam
    "STN", #Stansted
    "LTN", #Luton
    "LGW", #Gatwick
    "MAN", #Manchester
    "DUB", #Dublin
    "AYT", # Antalya
    "ADB", #Izmir
    "GLA", #Glasgow
    "EDI", #Edingburgh



]



In [75]:
#Creating List of Routes
import pandas as pd

# Load excluded routes
excluded_df = pd.read_excel("routes_to_exclude.xlsx")

# Create a set of tuples: (departure_id, arrival_id)
excluded_routes = set(
    tuple(route.split("-")) for route in excluded_df["route"]
)

# Initialize routes list
routes = []

# FROM Poland → Summer Destinations
for departure in polish_airports:
    for arrival in summer_destinations_from_poland:
        if (departure.upper(), arrival.upper()) not in excluded_routes:
            routes.append({
                "departure_id": departure,
                "arrival_id": arrival
            })

# FROM Summer Destinations → Poland
for departure in summer_destinations_from_poland:
    for arrival in polish_airports:
        if (departure.upper(), arrival.upper()) not in excluded_routes:
            routes.append({
                "departure_id": departure,
                "arrival_id": arrival
            })


print(f"Total number of routes (API requests): {len(routes)}")




Total number of routes (API requests): 1187


In [76]:
import requests
import pandas as pd

url = "https://google-flights2.p.rapidapi.com/api/v1/getCalendarPicker"


headers = {
	"x-rapidapi-key": "0ff57577b8msh7037b2fef615d77p119a7djsn50e7861599be",
	"x-rapidapi-host": "google-flights2.p.rapidapi.com"
}


# Store all results
all_results = []

# Query API for each route
for route in routes:
    params = {
        "departure_id": route["departure_id"],
        "arrival_id": route["arrival_id"],
        "travel_class": "ECONOMY",
        "trip_type": "ONE_WAY",
        "adults": "1",
        "currency": "PLN",
        "country_code": "PL",
        "end_date":"2025-10-31"
    }

    response = requests.get(url, headers=headers, params=params, verify=False)
    
    if response.status_code == 200:
        data = response.json()
        for entry in data.get("data", []):
            entry["departure_airport"] = route["departure_id"]
            entry["arrival_airport"] = route["arrival_id"]
            all_results.append(entry)
    else:
        print(f"Failed for {route['departure_id']} to {route['arrival_id']}")

# Convert to DataFrame
df = pd.DataFrame(all_results)

# Optional: reorder columns
df = df[["departure", "return", "price", "departure_airport", "arrival_airport"]]

# Print DataFrame
print(df)

df.to_excel("flight_prices_all.xlsx", index=False)


C:\Users\GADZINDA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'google-flights2.p.rapidapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\GADZINDA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'google-flights2.p.rapidapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\GADZINDA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ur

        departure return   price departure_airport arrival_airport
0      2025-08-01   None   992.0               WAW             HER
1      2025-08-02   None  1375.0               WAW             HER
2      2025-08-03   None   992.0               WAW             HER
3      2025-08-04   None  1210.0               WAW             HER
4      2025-08-05   None   719.0               WAW             HER
...           ...    ...     ...               ...             ...
90929  2025-10-27   None   304.0               EDI             POZ
90930  2025-10-28   None   516.0               EDI             POZ
90931  2025-10-29   None   516.0               EDI             POZ
90932  2025-10-30   None   176.0               EDI             POZ
90933  2025-10-31   None   182.0               EDI             POZ

[90934 rows x 5 columns]


In [77]:
import pandas as pd

# Define data as a list of dictionaries
airport_data = [
    # Poland
    {"IATA": "WAW", "City": "Warsaw", "Country": "Poland"},
    {"IATA": "WMI", "City": "Warsaw", "Country": "Poland"},
    {"IATA": "KRK", "City": "Krakow", "Country": "Poland"},
    {"IATA": "GDN", "City": "Gdansk", "Country": "Poland"},
    {"IATA": "WRO", "City": "Wroclaw", "Country": "Poland"},
    {"IATA": "KTW", "City": "Katowice", "Country": "Poland"},
    {"IATA": "POZ", "City": "Poznan", "Country": "Poland"},
    {"IATA": "RZE", "City": "Rzeszow", "Country": "Poland"},
    {"IATA": "LUZ", "City": "Lublin", "Country": "Poland"},


    # Greece
    {"IATA": "HER", "City": "Heraklion", "Country": "Greece"},
    {"IATA": "RHO", "City": "Rhodes", "Country": "Greece"},
    {"IATA": "SKG", "City": "Thessaloniki", "Country": "Greece"},
    {"IATA": "CFU", "City": "Corfu", "Country": "Greece"},
    {"IATA": "ZTH", "City": "Zakynthos", "Country": "Greece"},
    {"IATA": "KGS", "City": "Kos", "Country": "Greece"},
    {"IATA": "CHQ", "City": "Chania", "Country": "Greece"},

    # Spain
    {"IATA": "MAD", "City": "Madrid", "Country": "Spain"},
    {"IATA": "BCN", "City": "Barcelona", "Country": "Spain"},
    {"IATA": "AGP", "City": "Malaga", "Country": "Spain"},
    {"IATA": "PMI", "City": "Palma de Mallorca", "Country": "Spain"},
    {"IATA": "ALC", "City": "Alicante", "Country": "Spain"},
    {"IATA": "TFS", "City": "Tenerife", "Country": "Spain"},
    {"IATA": "SVQ", "City": "Seville", "Country": "Spain"},
    {"IATA": "IBZ", "City": "Ibiza", "Country": "Spain"},
    {"IATA": "VLC", "City": "Valencia", "Country": "Spain"},
    {"IATA": "BIO", "City": "Bilbao", "Country": "Spain"},
    {"IATA": "TFN", "City": "Tenerife", "Country": "Spain"},
    {"IATA": "SPC", "City": "La Palma", "Country": "Spain"},
    {"IATA": "GRX", "City": "Granada", "Country": "Spain"},

    # Italy
    {"IATA": "FCO", "City": "Rome", "Country": "Italy"},
    {"IATA": "CIA", "City": "Rome", "Country": "Italy"},
    {"IATA": "MXP", "City": "Milan", "Country": "Italy"},
    {"IATA": "BGY", "City": "Milan", "Country": "Italy"},
    {"IATA": "VCE", "City": "Venice", "Country": "Italy"},
    {"IATA": "NAP", "City": "Naples", "Country": "Italy"},
    {"IATA": "BLQ", "City": "Bologna", "Country": "Italy"},
    {"IATA": "PSA", "City": "Pisa", "Country": "Italy"},
    {"IATA": "FLR", "City": "Florence", "Country": "Italy"},
    {"IATA": "TRN", "City": "Turin", "Country": "Italy"},
    {"IATA": "BRI", "City": "Bari", "Country": "Italy"},
    {"IATA": "PMO", "City": "Palermo", "Country": "Italy"},
    {"IATA": "CTA", "City": "Catania", "Country": "Italy"},
    {"IATA": "OLB", "City": "Olbia", "Country": "Italy"},
    {"IATA": "AHO", "City": "Alghero", "Country": "Italy"},
    {"IATA": "VRN", "City": "Verona", "Country": "Italy"},
    {"IATA": "GOA", "City": "Genoa", "Country": "Italy"},

    # Croatia
    {"IATA": "SPU", "City": "Split", "Country": "Croatia"},
    {"IATA": "DBV", "City": "Dubrovnik", "Country": "Croatia"},
    {"IATA": "ZAD", "City": "Zadar", "Country": "Croatia"},

    # Cyprus
    {"IATA": "LCA", "City": "Larnaca", "Country": "Cyprus"},
    {"IATA": "PFO", "City": "Paphos", "Country": "Cyprus"},

    # Bulgaria
    {"IATA": "VAR", "City": "Varna", "Country": "Bulgaria"},
    {"IATA": "BOJ", "City": "Burgas", "Country": "Bulgaria"},
    {"IATA": "SOF", "City": "Sofia", "Country": "Bulgaria"},

    # Montenegro
    {"IATA": "TGD", "City": "Podgorica", "Country": "Montenegro"},

    # Albania
    {"IATA": "TIA", "City": "Tirana", "Country": "Albania"},

    # Portugal
    {"IATA": "FAO", "City": "Faro", "Country": "Portugal"},
    {"IATA": "LIS", "City": "Lisbon", "Country": "Portugal"},
    {"IATA": "OPO", "City": "Porto", "Country": "Portugal"},
    {"IATA": "FNC", "City": "Madeira", "Country": "Portugal"},

    # France
    {"IATA": "CDG", "City": "Paris", "Country": "France"},
    {"IATA": "ORY", "City": "Paris", "Country": "France"},
    {"IATA": "BVA", "City": "Paris", "Country": "France"},
    {"IATA": "LYS", "City": "Lyon", "Country": "France"},
    {"IATA": "NCE", "City": "Nice", "Country": "France"},
    {"IATA": "MRS", "City": "Marseille", "Country": "France"},
    {"IATA": "BIQ", "City": "Biarritz", "Country": "France"},

    # Iceland
    {"IATA": "KEF", "City": "Reykjavik", "Country": "Iceland"},

    # Other
    {"IATA": "RMO", "City": "Chisinau", "Country": "Moldova"},
    {"IATA": "BBU", "City": "Bucharest", "Country": "Romania"},
    {"IATA": "OTP", "City": "Bucharest", "Country": "Romania"},
    {"IATA": "CPH", "City": "Copenhagen", "Country": "Denmark"},
    {"IATA": "ARN", "City": "Stockholm", "Country": "Sweden"},
    {"IATA": "GOT", "City": "Gothenburg", "Country": "Sweden"},
    {"IATA": "ATH", "City": "Athens", "Country": "Greece"},
    {"IATA": "BUD", "City": "Budapest", "Country": "Hungary"},
    {"IATA": "AUH", "City": "Abu Dhabi", "Country": "United Arab Emirates"},
    {"IATA": "IST", "City": "Istanbul", "Country": "Turkey"},
    {"IATA": "SKP", "City": "Skopje", "Country": "North Macedonia"},
    {"IATA": "KUT", "City": "Kutaisi", "Country": "Georgia"},
    {"IATA": "MLA", "City": "Valletta", "Country": "Malta"},
    {"IATA": "AMM", "City": "Amman", "Country": "Jordan"},
    {"IATA": "RAK", "City": "Marrakesh", "Country": "Morocco"},
    {"IATA": "AGA", "City": "Agadir", "Country": "Morocco"},
    {"IATA": "RBA", "City": "Rabat", "Country": "Morocco"},
    {"IATA": "EIN", "City": "Eindhoven", "Country": "Netherlands"},
    {"IATA": "AMS", "City": "Amsterdam", "Country": "Netherlands"},
    {"IATA": "OSL", "City": "Oslo", "Country": "Norway"},
    {"IATA": "TRF", "City": "Oslo", "Country": "Norway"},
    {"IATA": "STN", "City": "London", "Country": "United Kingdom"},
    {"IATA": "LTN", "City": "London", "Country": "United Kingdom"},
    {"IATA": "LGW", "City": "London", "Country": "United Kingdom"},
    {"IATA": "MAN", "City": "Manchester", "Country": "United Kingdom"},
    {"IATA": "DUB", "City": "Dublin", "Country": "Ireland"},
    {"IATA": "AYT", "City": "Antalya", "Country": "Turkey"},
    {"IATA": "ADB", "City": "Izmir", "Country": "Turkey"},
    {"IATA": "GLA", "City": "Glasgow", "Country": "Scotland"},
    {"IATA": "EDI", "City": "Edinburgh", "Country": "Scotland"},



]

# Create DataFrame
airport_df = pd.DataFrame(airport_data)

# Preview
print(airport_df.head())


  IATA     City Country
0  WAW   Warsaw  Poland
1  WMI   Warsaw  Poland
2  KRK   Krakow  Poland
3  GDN   Gdansk  Poland
4  WRO  Wroclaw  Poland


In [78]:
import pandas as pd
from datetime import datetime

# Add today's date to a new column
df['date_of_export'] = pd.to_datetime('today').normalize()

# Merge to get DepartureCity and DepartureCountry
df = df.merge(
    airport_df.rename(columns={"IATA": "departure_airport", "City": "DepartureCity", "Country": "DepartureCountry"}),
    on="departure_airport",
    how="left"
)

# Merge to get ArrivalCity and ArrivalCountry
df = df.merge(
    airport_df.rename(columns={"IATA": "arrival_airport", "City": "ArrivalCity", "Country": "ArrivalCountry"}),
    on="arrival_airport",
    how="left"
)

# Create filename with today's date in DDMMYYYY format
today_str = datetime.today().strftime('%d%m%Y')
filename = f"flight_prices_{today_str}.xlsx"

# Save to Excel
df.to_excel(filename, index=False)

print(f"DataFrame saved to {filename}")


DataFrame saved to flight_prices_01082025.xlsx


In [79]:
# Load the Excel file into a DataFrame
df = pd.read_excel(filename)

# Check the first few rows to verify
print(df.head())


    departure  return   price departure_airport arrival_airport  \
0  2025-08-01     NaN   992.0               WAW             HER   
1  2025-08-02     NaN  1375.0               WAW             HER   
2  2025-08-03     NaN   992.0               WAW             HER   
3  2025-08-04     NaN  1210.0               WAW             HER   
4  2025-08-05     NaN   719.0               WAW             HER   

  date_of_export DepartureCity DepartureCountry ArrivalCity ArrivalCountry  
0     2025-08-01        Warsaw           Poland   Heraklion         Greece  
1     2025-08-01        Warsaw           Poland   Heraklion         Greece  
2     2025-08-01        Warsaw           Poland   Heraklion         Greece  
3     2025-08-01        Warsaw           Poland   Heraklion         Greece  
4     2025-08-01        Warsaw           Poland   Heraklion         Greece  


In [80]:
import pandas as pd
from datetime import datetime

# Ensure datetime format
df['departure'] = pd.to_datetime(df['departure'])
df['return'] = pd.to_datetime(df['return'])

# Rename columns to make merging easier
df = df.rename(columns={
    'departure_airport': 'DepartureAirport',
    'arrival_airport': 'ArrivalAirport',
    'DepartureCity': 'DepartureCity',
    'ArrivalCity': 'ArrivalCity',
    'DepartureCountry': 'DepartureCountry',
    'ArrivalCountry': 'ArrivalCountry'
})

# Create outbound and inbound datasets
outbound = df[df['DepartureCountry'] == 'Poland'].copy()
inbound = df[df['ArrivalCountry'] == 'Poland'].copy()

# Rename inbound columns to avoid collisions when merging
inbound = inbound.rename(columns={
    'DepartureAirport': 'ReturnDepartureAirport',
    'ArrivalAirport': 'ReturnArrivalAirport',
    'DepartureCity': 'ReturnDepartureCity',
    'ArrivalCity': 'ReturnArrivalCity',
    'DepartureCountry': 'ReturnDepartureCountry',
    'ArrivalCountry': 'ReturnArrivalCountry',
    'departure': 'return_date',
    'price': 'return_price'
})

# Merge on destination city (city you go to = city you return from)
merged = pd.merge(
    outbound,
    inbound,
    left_on='ArrivalCity',
    right_on='ReturnDepartureCity',
    suffixes=('_out', '_in')
)

# Calculate trip duration
merged['trip_duration_days'] = (merged['return_date'] - merged['departure']).dt.days

# Filter for trips between 2 and 7 days
valid_trips = merged[
    (merged['trip_duration_days'] >= 2) & 
    (merged['trip_duration_days'] <= 7)
].copy()

# Calculate total price
valid_trips['total_price'] = valid_trips['price'] + valid_trips['return_price']

# Extract month for grouping
valid_trips['departure_month'] = valid_trips['return_date'].dt.to_period('M')

# Create route identifier
valid_trips['route'] = valid_trips['DepartureCity'] + ' - ' + valid_trips['ArrivalCity']

# Group by route and departure month, get top 10% cheapest flights per group
def top_10_percent(group):
    cutoff = int(len(group) * 0.2)
    if cutoff == 0:
        cutoff = 1
    return group.nsmallest(cutoff, 'total_price')

filtered_df = (
    valid_trips
    .groupby(['route', 'departure_month'], group_keys=False)
    .apply(top_10_percent)
)

# Final columns
final_df = filtered_df[[
    'DepartureCity', 'ArrivalCity', 'ArrivalCountry', 'ReturnArrivalCity',
    'departure', 'return_date',
    'trip_duration_days', 'price', 'return_price', 'total_price', 'DepartureAirport', 'ReturnDepartureAirport', 'ReturnArrivalAirport'
]].rename(columns={
    'DepartureCity': 'Outbound From',
    'ArrivalCity': 'Destination',
    'ReturnArrivalCity': 'Inbound To',
    'departure': 'Departure Date',
    'return_date': 'Return Date',
    'price': 'Outbound Price',
    'return_price': 'Inbound Price',
    'total_price': 'Total Price',
    'trip_duration_days': 'Trip Duration (Days)',
    'DepartureAirport': 'IATA_Departure',
    'ReturnDepartureAirport': 'IATA_Destination',
    'ReturnArrivalAirport': 'IATA_Return'
})

# Sort by Total Price
final_df = final_df.sort_values(by='Total Price').reset_index(drop=True)

import pandas as pd
from datetime import datetime, timedelta



# Updated function (now works with pd.Timestamp or datetime.datetime)
def format_date(dt):
    dt_adjusted = dt
    return dt_adjusted.strftime('%y%m%d')

# Create outbound link:
final_df['Outbound_Link'] = final_df.apply(
    lambda row: f"https://www.skyscanner.pl/transport/loty/{row['IATA_Departure']}/{row['IATA_Destination']}/{format_date(row['Departure Date'])}/"
                "?"
                "adultsv2=1&cabinclass=economy&childrenv2=&inboundaltsenabled=false&outboundaltsenabled=false&preferdirects=false&ref=home&rtn=0",
    axis=1
)

# Create inbound link:
final_df['Inbound_Link'] = final_df.apply(
    lambda row: f"https://www.skyscanner.pl/transport/loty/{row['IATA_Destination']}/{row['IATA_Return']}/{format_date(row['Return Date'])}/"
                "?"
                "adultsv2=1&cabinclass=economy&childrenv2=&inboundaltsenabled=false&outboundaltsenabled=false&preferdirects=false&ref=home&rtn=0",
    axis=1
)

# Create round trip link (only if departure = return airport)
final_df['Round_Trip_Link'] = final_df.apply(
    lambda row: (
        f"https://www.skyscanner.pl/transport/loty/{row['IATA_Departure']}/{row['IATA_Destination']}/"
        f"{format_date(row['Departure Date'])}/{format_date(row['Return Date'])}/"
        "?"
        "adultsv2=1&cabinclass=economy&childrenv2=&ref=home&rtn=1&preferdirects=false&outboundaltsenabled=false&inboundaltsenabled=false"
    ) if row['IATA_Departure'] == row['IATA_Return'] else None,
    axis=1
)



# Export to Excel
today_str = datetime.today().strftime('%d%m%Y')

filename2 = f"multi_city_tickets_{today_str}.csv"
final_df.to_csv(filename2, index=False, encoding='utf-8-sig')


print("Filtered dataframe with top 10% cheapest round-trips by route and month saved.")



Filtered dataframe with top 10% cheapest round-trips by route and month saved.


In [81]:
#top 5 CITY BREAKS
import pandas as pd
from datetime import datetime, timedelta

# === STEP 1: Load the CSV ===
df = pd.read_csv(filename2)


# === STEP 2: Convert dates ===
df["Departure Date"] = pd.to_datetime(df["Departure Date"], dayfirst=True, errors='coerce')
df["Return Date"] = pd.to_datetime(df["Return Date"], dayfirst=True, errors='coerce')

# === STEP 3: Generate next 4 weekends (only Fri->Sun and Sat->Mon pairs) ===
today = datetime.today()
weekend_pairs = []

for i in range(1, 6):  # Next 4 weekends
    base_day = today + timedelta(days=(4 - today.weekday()) % 7 + (i - 1) * 7)  # Friday
    friday = base_day.date()
    saturday = (base_day + timedelta(days=1)).date()
    sunday = (base_day + timedelta(days=2)).date()
    monday = (base_day + timedelta(days=3)).date()
    # Only keep these two pairs:
    weekend_pairs.append((friday, sunday))   # Fri -> Sun
    weekend_pairs.append((saturday, monday)) # Sat -> Mon

# === STEP 4: Filter trips that match weekend departure/return ===
def is_weekend_trip(row):
    dep = row["Departure Date"].date()
    ret = row["Return Date"].date()
    return (dep, ret) in weekend_pairs

df_filtered = df[df.apply(is_weekend_trip, axis=1)].copy()
df_filtered = df_filtered[df_filtered["Outbound From"] == df_filtered["Inbound To"]].copy()


# === Step 5: Format columns for output ===
df_filtered["DATA WYLOTU"] = df_filtered["Departure Date"].dt.strftime("%d.%m")
df_filtered["DATA PRZYLOTU"] = df_filtered["Return Date"].dt.strftime("%d.%m")
df_filtered["CENA"] = df_filtered["Total Price"].astype("Int64").astype(str) + " ZŁ"
df_filtered["LICZBA DNI"] = df_filtered["Trip Duration (Days)"]

# === Step 6: Get unique Polish departure cities ===
polish_cities = df_filtered["Outbound From"].dropna().unique()

# === Step 7: Export to Excel with one sheet per city, top 5 cheapest ===
output_path = "top_5_weekend_city_breaks.xlsx"
# Inside the export loop, replace city_df selection with unique destinations only

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    for city in polish_cities:
        city_df = df_filtered[df_filtered["Outbound From"] == city]
        
        # Sort by price ascending
        city_df = city_df.sort_values("Total Price")
        
        # Keep only first occurrence of each destination city (unique destinations)
        city_df_unique = city_df.drop_duplicates(subset=["Destination"])
        
        # Take top 5 cheapest unique destinations
        top5 = city_df_unique.head(5)
        
        top5[[
            "Outbound From", "Destination", "DATA WYLOTU", "DATA PRZYLOTU", 
            "LICZBA DNI", "CENA", "Outbound_Link", "Inbound_Link", "Round_Trip_Link"
        ]].to_excel(writer, sheet_name=city[:31], index=False)


print(f"✅ Excel saved as {output_path}")

✅ Excel saved as top_5_weekend_city_breaks.xlsx


In [82]:
#best deals
import pandas as pd
from datetime import datetime

# Load data
df = pd.read_csv(filename2)

# Ensure date format
df['Departure Date'] = pd.to_datetime(df['Departure Date'], errors='coerce')

# Create route and departure month
df['Route'] = df['Outbound From'] + '-' + df['Destination']
df['Month'] = df['Departure Date'].dt.to_period('M')

# Drop rows with missing or invalid price data
df = df.dropna(subset=['Total Price', 'Departure Date'])
df['Total Price'] = pd.to_numeric(df['Total Price'], errors='coerce')
df = df.dropna(subset=['Total Price'])

# Group by Route and Month to compute stats
stats = df.groupby(['Route', 'Month'])['Total Price'].agg(['mean', 'std']).reset_index()
stats.rename(columns={'mean': 'AvgPrice', 'std': 'StdDev'}, inplace=True)

# Merge statistics back into main DataFrame
df = df.merge(stats, on=['Route', 'Month'], how='left')

# Calculate z-score
df['z_score'] = (df['Total Price'] - df['AvgPrice']) / df['StdDev']

# Filter best deals: at least 1 std dev below avg
best_deals = df[df['z_score'] <= -1].copy()

# Sort by best deals first
best_deals = best_deals.sort_values(by='z_score')

# Save to Excel
today = datetime.today().strftime('%d%m%Y')
output_filename = f"best_deals_detected_{today}.csv"
best_deals.to_csv(output_filename, index=False)

print(f"✅ Saved {len(best_deals)} best deals to '{output_filename}'")


✅ Saved 67822 best deals to 'best_deals_detected_01082025.csv'


In [83]:
import pandas as pd

# Load your final_df
final_df = pd.read_csv(filename2)

# Ensure correct dtypes
final_df["Total Price"] = final_df["Total Price"].astype(float)
final_df["Departure Date"] = pd.to_datetime(final_df["Departure Date"])
final_df["Return Date"] = pd.to_datetime(final_df["Return Date"])

# Filter: Only flights returning by end of September
final_df = final_df[final_df["Return Date"] <= pd.Timestamp("2025-09-30")]

# Format output for Excel
def format_output(df):
    df = df.copy()
    df["KIERUNEK"] = "➡️ " + df["Destination"]
    df["DATA WYLOTU"] = df["Departure Date"].dt.strftime("%d.%m")
    df["DATA PRZYLOTU"] = df["Return Date"].dt.strftime("%d.%m")
    df["ILOŚĆ DNI"] = (df["Return Date"] - df["Departure Date"]).dt.days
    df["CENA"] = df["Total Price"].astype(int).astype(str) + " ZŁ"
    return df[["KIERUNEK", "DATA WYLOTU", "DATA PRZYLOTU", "ILOŚĆ DNI", "CENA"]]

# Get all unique Polish cities
polish_cities = final_df["Outbound From"].unique()

# Writers
writer_same_city = pd.ExcelWriter("Top10_SameCity_Flights.xlsx", engine="openpyxl")
writer_diff_return = pd.ExcelWriter("Top10_MixedReturn_Flights.xlsx", engine="openpyxl")

# Process each city
for city in polish_cities:
    # === SAME CITY ===
    df_city = final_df[
        (final_df["Outbound From"] == city) & (final_df["Inbound To"] == city)
    ].copy()

    df_unique_cities = (
        df_city.sort_values("Total Price")
        .drop_duplicates(subset=["Destination"])
        .groupby("Outbound From")
        .head(10)
    )

    df_unique_countries = (
        df_city.sort_values("Total Price")
        .drop_duplicates(subset=["ArrivalCountry"])
        .groupby("Outbound From")
        .head(10)
    )

    # ✅ FORMAT before saving
    format_output(df_unique_cities).to_excel(writer_same_city, sheet_name=f"{city}_Cities", index=False)
    format_output(df_unique_countries).to_excel(writer_same_city, sheet_name=f"{city}_Countries", index=False)

    # === MIXED RETURN ===
    df_mixed = final_df[final_df["Outbound From"] == city].copy()

    df_mixed_cities = (
        df_mixed.sort_values("Total Price")
        .drop_duplicates(subset=["Destination"])
        .groupby("Outbound From")
        .head(10)
    )

    df_mixed_countries = (
        df_mixed.sort_values("Total Price")
        .drop_duplicates(subset=["ArrivalCountry"])
        .groupby("Outbound From")
        .head(10)
    )

    # ✅ FORMAT before saving
    format_output(df_mixed_cities).to_excel(writer_diff_return, sheet_name=f"{city}_Cities", index=False)
    format_output(df_mixed_countries).to_excel(writer_diff_return, sheet_name=f"{city}_Countries", index=False)

# Save
writer_same_city.close()
writer_diff_return.close()

print("✅ Excel files created successfully with correct formatting.")


✅ Excel files created successfully with correct formatting.


In [84]:
import os
import shutil
import glob

# Paths
current_dir = os.getcwd()
archive_dir = os.path.join(current_dir, "ARCHIVE")
pbi_dir = os.path.join(current_dir, "PBI")

# Ensure destination folders exist
os.makedirs(archive_dir, exist_ok=True)
os.makedirs(pbi_dir, exist_ok=True)

# Step 1: Move best_deals_detected_*.csv and multi_city_tickets_*.csv from PBI to ARCHIVE
for pattern in ["best_deals_detected_*.csv", "multi_city_tickets_*.csv"]:
    for file in glob.glob(os.path.join(pbi_dir, pattern)):
        shutil.move(file, os.path.join(archive_dir, os.path.basename(file)))

# Step 2: Move all CSV files to PBI
for csv_file in glob.glob(os.path.join(current_dir, "*.csv")):
    shutil.move(csv_file, os.path.join(pbi_dir, os.path.basename(csv_file)))

# Step 3: Move flight_prices to ARCHIVE
for pattern in ["flight_prices_*.xlsx"]:
    for file in glob.glob(os.path.join(archive_dir, pattern)):
        shutil.move(file, os.path.join(pbi_dir, os.path.basename(file)))

print("✅ Files moved successfully.")


✅ Files moved successfully.
